# CAPM & Simple Portfolio Demo (Public Data)

This notebook recreates the core of my finance coursework using **public data** instead of Bloomberg: returns, a 2-asset portfolio sweep, and a CAPM regression.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

tick_stock = 'HSBC.L'   # single stock example
tick_mkt   = '^FTSE'    # FTSE 100 proxy
start, end = '2020-01-01', '2023-01-01'

prices = yf.download([tick_stock, tick_mkt], start=start, end=end)['Adj Close'].dropna()
prices.head()

In [ ]:
# Daily returns
rets = prices.pct_change().dropna()
stock_ret = rets[tick_stock]
mkt_ret   = rets[tick_mkt]

# Assume rf = 0 for demo
excess_stock = stock_ret
excess_mkt   = mkt_ret

rets.describe()

In [ ]:
# Plot stock vs market returns
rets[[tick_stock, tick_mkt]].plot(title=f"Daily Returns: {tick_stock} vs {tick_mkt}")
plt.xlabel('Date'); plt.ylabel('Return'); plt.show()

## 2-Asset Portfolio Sweep
Weights for stock go from 0 to 1, market gets the rest.

In [ ]:
w_grid = np.linspace(0, 1, 21)
mu = rets.mean() * 252  # annualised mean
cov = rets.cov() * 252  # annualised covariance

mus, sigmas = [], []
for w in w_grid:
    weights = np.array([w, 1-w])
    mu_p = weights @ mu[[tick_stock, tick_mkt]].values
    sigma_p = np.sqrt(weights @ cov.loc[[tick_stock, tick_mkt], [tick_stock, tick_mkt]].values @ weights)
    mus.append(mu_p); sigmas.append(sigma_p)

plt.plot(sigmas, mus, marker='o')
plt.title('Risk–Return Sweep (HSBC vs FTSE)')
plt.xlabel('Annualised Volatility'); plt.ylabel('Annualised Return')
plt.show()

## CAPM Regression
$r_{stock} - r_f = \alpha + \beta (r_{mkt} - r_f) + \varepsilon$

In [ ]:
X = sm.add_constant(excess_mkt.values)
y = excess_stock.values
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
# Scatter with fitted line
plt.scatter(excess_mkt, excess_stock, alpha=0.4)
beta, alpha = model.params[1], model.params[0]
xline = np.linspace(excess_mkt.min(), excess_mkt.max(), 100)
yline = alpha + beta * xline
plt.plot(xline, yline, color='red')
plt.title(f'CAPM: {tick_stock} vs {tick_mkt} (beta={beta:.2f}, alpha={alpha:.4f})')
plt.xlabel('Market Excess Return'); plt.ylabel('Stock Excess Return')
plt.show()